In [318]:
import random
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [319]:
class Layer():
    def __init__(self):
        self.params = []
        self.grads = []

    def __call__(self, x):
        # por defecto, devolver los inputs
        # cada capa hará algo diferente aquí
        return x

    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    def update(self, params):
        # si hay parámetros, los actualizaremos
        # con lo que nos de el optimizer
        return

In [320]:
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # pesos de la capa
        self.w = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(d_in+d_out)),
                                  size=(d_in, d_out))
        self.b = np.zeros(d_out)

    def __call__(self, x):
        self.x = x
        self.params = [self.w, self.b]
        # salida del preceptrón
        return np.dot(x, self.w) + self.b

    def backward(self, grad_output):
        # gradientes para la capa siguiente (BACKPROP)
        grad = np.dot(grad_output, self.w.T)
        self.grad_w = np.dot(self.x.T, grad_output)
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params):
        self.w, self.b = params

In [321]:
#funciones de activacion
class ReLU(Layer):
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, grad_output):
        grad = self.x > 0
        return grad_output*grad

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)

    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x))
        return grad_output*grad

In [322]:
#Optimizador, stocastic greadent descent
class SGD():
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def update(self):
        for layer in self.net.layers:
            layer.update([
                params - self.lr*grads
                for params, grads in zip(layer.params, layer.grads)
            ])

In [323]:
class Loss():
    def __init__(self, net):
        self.net = net

    def backward(self):
        # derivada de la loss function con respecto
        # a la salida del MLP
        grad = self.grad_loss()
        # BACKPROPAGATION
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)

class MSE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = np.mean((self.output - self.target)**2)
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class BCE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = - np.mean(self.target*np.log(self.output) - (1 - self.target)*np.log(1 - self.output))
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class CrossEntropy(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
        loss = loss.mean()
        return loss

    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target] = 1
        return (- answers + softmax(self.output)) / self.output.shape[0]

In [324]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

#Carga de dataset
data = pd.read_csv('/content/car_price_prediction.csv')

In [325]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  object 
 3   Manufacturer      19237 non-null  object 
 4   Model             19237 non-null  object 
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  object 
 7   Leather interior  19237 non-null  object 
 8   Fuel type         19237 non-null  object 
 9   Engine volume     19237 non-null  object 
 10  Mileage           19237 non-null  object 
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  object 
 13  Drive wheels      19237 non-null  object 
 14  Doors             19237 non-null  object 
 15  Wheel             19237 non-null  object 
 16  Color             19237 non-null  object

In [326]:
columnas_categoricas = data.select_dtypes(include=['object']).columns
columnas_fechas = data.select_dtypes(include=['datetime64']).columns

#Procesamiento de frases
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

#Fechas a numeros
for columna in columnas_fechas:
  data[columna] = pd.to_numeric(data[columna].map(datetime.timestamp))

In [327]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  int64  
 3   Manufacturer      19237 non-null  int64  
 4   Model             19237 non-null  int64  
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  int64  
 7   Leather interior  19237 non-null  int64  
 8   Fuel type         19237 non-null  int64  
 9   Engine volume     19237 non-null  int64  
 10  Mileage           19237 non-null  int64  
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  int64  
 13  Drive wheels      19237 non-null  int64  
 14  Doors             19237 non-null  int64  
 15  Wheel             19237 non-null  int64  
 16  Color             19237 non-null  int64 

In [328]:
columnas_eiliminar = ['ID']
data = data.drop(columnas_eiliminar, axis=1)

In [329]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             19237 non-null  int64  
 1   Levy              19237 non-null  int64  
 2   Manufacturer      19237 non-null  int64  
 3   Model             19237 non-null  int64  
 4   Prod. year        19237 non-null  int64  
 5   Category          19237 non-null  int64  
 6   Leather interior  19237 non-null  int64  
 7   Fuel type         19237 non-null  int64  
 8   Engine volume     19237 non-null  int64  
 9   Mileage           19237 non-null  int64  
 10  Cylinders         19237 non-null  float64
 11  Gear box type     19237 non-null  int64  
 12  Drive wheels      19237 non-null  int64  
 13  Doors             19237 non-null  int64  
 14  Wheel             19237 non-null  int64  
 15  Color             19237 non-null  int64  
 16  Airbags           19237 non-null  int64 

In [330]:
print(data)

       Price  Levy  Manufacturer  Model  Prod. year  Category  \
0      13328   114            32   1242        2010         4   
1      16621     4             8    658        2011         4   
2       8467     0            21    684        2006         3   
3       3607   512            16    661        2011         4   
4      11726   366            21    684        2014         3   
...      ...   ...           ...    ...         ...       ...   
19232   8467     0            36    385        1999         1   
19233  15681   504            23   1334        2011         9   
19234  26108   508            23   1442        2010         4   
19235   5331    83             8    456        2007         4   
19236    470   480            23   1334        2012         9   

       Leather interior  Fuel type  Engine volume  Mileage  Cylinders  \
0                     1          2             63     2838        6.0   
1                     0          5             56     2960        6.0   


In [331]:
x = data.iloc[:, 1:17]
y = data.iloc[:, 0]
m = y.size
print(x.shape)
print(y.shape)
print(x)
print('---'*20)
print(y)

(19237, 16)
(19237,)
       Levy  Manufacturer  Model  Prod. year  Category  Leather interior  \
0       114            32   1242        2010         4                 1   
1         4             8    658        2011         4                 0   
2         0            21    684        2006         3                 0   
3       512            16    661        2011         4                 1   
4       366            21    684        2014         3                 1   
...     ...           ...    ...         ...       ...               ...   
19232     0            36    385        1999         1                 1   
19233   504            23   1334        2011         9                 1   
19234   508            23   1442        2010         4                 1   
19235    83             8    456        2007         4                 1   
19236   480            23   1334        2012         9                 1   

       Fuel type  Engine volume  Mileage  Cylinders  Gear box type

In [332]:
# Crea un nuevo DataFrame con los datos modificados
nuevo_data = data.copy()
# Guardar el dataset actualizado en un nuevo archivo
nuevo_data.to_csv('/content/car2_price_prediction.csv', index=False)

In [333]:
data = np.loadtxt("/content/car2_price_prediction.csv", delimiter=',',skiprows=1)
# print(data)
X, Y = data[:, 1:17].astype(int), data[:, 0].astype(int)
X = X.reshape(len(X),16)
print(X.shape)
print(Y.shape)

(19237, 16)
(19237,)


In [334]:
X_train, X_test, y_train, y_test = X[:800] , X[800:] , Y[:800].astype(np.int64), y[800:].astype(np.int64)
X_train.shape , X_test.shape

y_train.shape , y_test.shape

((800,), (18437,))

In [335]:
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

In [336]:
def linear(x):
    return x

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

In [337]:
# Mean Square Error -> usada para regresión (con activación lineal)
def mse(y, y_hat):
    return np.mean((y_hat - y.reshape(y_hat.shape))**2)

# Binary Cross Entropy -> usada para clasificación binaria (con sigmoid)
def bce(y, y_hat):
    return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))

# Cross Entropy (aplica softmax + cross entropy de manera estable) -> usada para clasificación multiclase
def crossentropy(y, y_hat):
    logits = y_hat[np.arange(len(y_hat)),y]
    entropy = - logits + np.log(np.sum(np.exp(y_hat),axis=-1))
    return entropy.mean()

In [338]:
def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_bce(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_crossentropy(y, y_hat):
    answers = np.zeros_like(y_hat)
    answers[np.arange(len(y_hat)),y] = 1
    return (- answers + softmax(y_hat)) / y_hat.shape[0]

In [339]:
# clase base MLP

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2
    return self.activation(y_hat)

  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size]
            # salida del perceptrón
            y_pred = self(x)
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)
            # Backprop
            dldy = self.grad_loss(y, y_pred)
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)

In [340]:
# MLP para regresión
class MLPRegression(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, mse, grad_mse, linear)

# MLP para clasificación binaria
class MLPBinaryClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, bce, grad_bce, sigmoid)

# MLP para clasificación multiclase
class MLPClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, crossentropy, grad_crossentropy, linear)

In [341]:
model = MLPRegression(D_in=16, H=3, D_out=1)
epochs, lr = 60, 0.02

# normalización datos
x_mean = np.mean(X_train)
x_std =  np.std(X_train)
x_std = np.nan_to_num(x_std, nan=1.0)


# Calcula x_norm
x_norm = (X_train - x_mean) / x_std

model.fit(x_norm, y_train, epochs, lr, batch_size=1, log_each=10)

Epoch: 10/60, Loss: 294098188.98935
Epoch: 20/60, Loss: 293736390.67204
Epoch: 30/60, Loss: 293615791.23294
Epoch: 40/60, Loss: 293555491.51339
Epoch: 50/60, Loss: 293519311.68166
Epoch: 60/60, Loss: 293495191.79384


In [342]:
# últimos pesos encontrados

w = model.ws[-1]
w

(array([[-1.46337467e-01, -2.96190967e+00,  1.73942689e-01],
        [ 4.08595737e-02,  1.33178377e+01,  3.06934833e-01],
        [ 1.42609045e-01, -8.18962796e+00, -1.87030444e-01],
        [-2.59479851e-01, -5.58286064e+01, -4.53363628e-01],
        [ 3.14941871e-01,  1.43545771e+01,  3.21338889e-01],
        [-9.69157884e-02,  1.37786214e+01,  6.59307678e-02],
        [ 1.66032658e-01,  1.39760064e+01, -1.85070546e-01],
        [ 6.57459878e-02,  1.28234602e+01,  4.30557869e-01],
        [-4.04087363e-02, -1.50640378e+01, -1.63213592e-01],
        [ 2.28816784e-01,  1.36469113e+01, -9.21822090e-01],
        [ 2.23570945e-01,  1.40188677e+01,  5.09487297e-01],
        [ 7.74663037e-01,  1.42188693e+01, -1.16198104e-02],
        [-1.53826049e-01,  1.39523259e+01,  3.10917638e-01],
        [ 3.63168550e-01,  1.39026726e+01, -4.89036447e-01],
        [ 3.63018371e-01,  1.38376765e+01, -1.39562503e-01],
        [-2.23890429e-01,  1.28594628e+01,  2.50830959e-01]]),
 array([  0.        , 

In [343]:
print("Primeros 5 datos de entrada de prueba (x_test):")
print(X_test[:5])

print("Primeras 5 etiquetas de prueba (y_test):")
print(y_test[:5])

Primeros 5 datos de entrada de prueba (x_test):
[[ 554   21  526 2010    4    1    5   44  237    4    0    0    1    0
    12   12]
 [  18   36  601 2012    9    1    1   63 4544    6    0    0    1    0
     1   12]
 [   0    5 1533 2001    4    1    5   57 3752    6    2    0    1    0
     1    6]
 [ 528   23 1442 2016    4    1    1   36 7486    4    0    1    1    0
    14    4]
 [   0   41 1410 2005    3    0    5   26 7567    4    0    1    1    1
    12    0]]
Primeras 5 etiquetas de prueba (y_test):
800     8311
801      627
802    13485
803    38445
804     9722
Name: Price, dtype: int64


In [344]:
# nuevo punto
x_new = X_test
X_new = [554,21,526, 2010,4,1,5,44,237,4, 0, 0, 1, 0, 12, 12]
y_pred = model.predict(w, X_new)
y_pred

array([16680.67377385])

In [345]:
def evaluate(model,x, t = 0.5):
    w = model.ws[-1]
    x = np.c_[np.ones(len(X)), X]
    y = model.predict(w, x)
    return (y > t).astype(np.int64)

In [346]:
def accuracy(y_pred, y):
    return np.sum(y_pred == y) / len(y)